In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

In [ ]:
model_path = 'hand_landmark.task'

In [ ]:
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands


In [ ]:
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       running_mode=VisionRunningMode.VIDEO,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

In [ ]:
video_path = r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_DICTIONARY\I\ISL Dictionary\A\Alien_species.mp4"
cap = cv2.VideoCapture(video_path)

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
    detect=detector(mp_image)
    

In [ ]:
import mediapipe as mp
import cv2

# Initialize MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Use OpenCV's VideoCapture to load the input video
#cap = cv2.VideoCapture('path_to_your_video.mp4')

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Unable to open video.")
    exit()

# Get the frame rate of the video using OpenCV's CAP_PROP_FPS
fps = cap.get(cv2.CAP_PROP_FPS)

# Loop through each frame in the video
while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break  # Break the loop if there are no more frames

    # Convert the frame to RGB (MediaPipe requires RGB input)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert the frame to a MediaPipe's Image object
    mp_image = mp.Image(
        image_format=mp.ImageFormat.SRGB,
        data=frame_rgb#.tobytes()#,        width=frame_rgb.shape[1],        height=frame_rgb.shape[0],
    )

    # Process the frame using MediaPipe's Hand tracking
    with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hands:
        # Run hand detection on the frame
        results = hands.process(mp_image)

        # Draw hand landmarks on the frame
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the processed frame
    cv2.imshow('MediaPipe Hands', frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()


In [25]:
#@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [27]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
import cv2

# STEP 2: Create a HandLandmarker object.
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# STEP 3: Open the video file or webcam stream.
video_path = r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_DICTIONARY\I\ISL Dictionary\A\Alien_species.mp4"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    # STEP 4: Read frames from the video.
    ret, frame = cap.read()
    if not ret:
        break

    # STEP 5: Convert the BGR frame to RGB and process it for hand detection.
    rgb_frame1 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb_frame = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame1)
    #results = hands.process(rgb_frame)
    results = detector.detect(rgb_frame)

    # STEP 6: Process the detection result and annotate the image.
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            annotated_image = draw_landmarks_on_image(rgb_frame.numpy_view(), results)


    # STEP 7: Display the annotated frame.
    cv2.imshow('Hand Landmarks', frame)

    # Press 'q' to exit the loop.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# STEP 8: Release resources.
cap.release()
cv2.destroyAllWindows()


ValueError: Task is not initialized with the image mode. Current running mode:VIDEO